**determine who was**
* best dressed
* worst dressed 
* most controversial

In [112]:
import pandas as pd

df = pd.read_json(path_or_buf='gg2013.json')
df.head(10)

data = df['text']

In [113]:
import re
def cleanse(line):
    # replace everything to ' ' except whitespace, alphanumeric character, apostrophe, hashtag, @
    return re.sub(r'[^\w\s\'#@]', ' ', line)

In [114]:
import spacy
# python -m spacy download en
nlp = spacy.load('en')
doc = nlp(cleanse(data[0]))
print([(w.text, w.pos_) for w in doc])

[('JLo', 'PROPN'), ("'s", 'PART'), ('dress', 'NOUN'), (' ', 'SPACE'), ('#', 'SYM'), ('eredcarpet', 'NOUN'), ('#', 'SYM'), ('GoldenGlobes', 'PROPN')]


In [115]:
retweets_freq_dict = {}

def remove_retweet_prefix(line):
    # find 'RT @abc: ' where abc's length is arbitrary
    pattern = re.compile(r'\bRT @([\w\'/]*)\b: ') 
   
    match = re.search(pattern, line)
    if match:
        # store corresponding retweet without 'RT @' prefix
        string = match.group()[4:]
        if string in retweets_freq_dict:
            retweets_freq_dict[string] += 1
        else:
            retweets_freq_dict[string] = 1
        
    return re.sub(pattern, ' ', line)

In [116]:
def remove_apostrophe(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    return text

# test
print(remove_apostrophe("Tina 's"))

Tina


In [117]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [118]:
import pprint
def print_top(n, num):
    sorted_nominees = sorted(n.items(), key=lambda e: e[1], reverse=True)
    pprint.pprint(sorted_nominees[0:num])
#     names = [pair[0] for pair in sorted_nominees]
#     pprint.pprint(names[0:num])
    print('list length:', len(n))

In [119]:
at_freq_dict = {}

def remove_at(line):
    pattern = re.compile(r'@([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in at_freq_dict:
                at_freq_dict[match] += 1
            else:
                at_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

In [120]:
def remove_apostrophe(text):
    # remove_apostrophe
    if text.endswith("'s"):
        return text[:-2].strip()
    return text

# test
print(remove_apostrophe("Tina 's"))

Tina


In [121]:
hashtag_freq_dict = {}

def remove_hashtag(line):
    pattern = re.compile(r'#([\w\'/]*)\b')
    matches = re.findall(pattern, line)
    if matches:
        # store corresponding hashtag
        for match in matches:
            if match in hashtag_freq_dict:
                hashtag_freq_dict[match] += 1
            else:
                hashtag_freq_dict[match] = 1
        
    line = re.sub(pattern, ' ', line)
    return line

def remove_url(line):
    pattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b')
    matches = re.findall(pattern, line)
    for match in matches:
        line = re.sub(match, ' ', line)
    return line

In [122]:
from textblob import TextBlob
# import nltk
# nltk.download('punkt')
# pip3 install TextBlob

def get_polarity(line):
    blob = TextBlob(line)
    for sentence in blob.sentences:
#         print(sentence.sentiment.polarity)
        return float(sentence.sentiment.polarity)
    
line = "Jennifer Lopez's dress is jaw droppingly amazing #GoldenGlobes #redcarpet"
get_polarity(line)

0.6000000000000001

In [111]:
%%time
from fuzzywuzzy import fuzz
from textblob import TextBlob

def find_names():
    pattern = re.compile("(dress)|(wear)|(outfit)|(suit)|(cloth)", re.IGNORECASE)
    # pattern1 = re.compile("(\saward\s)|(\sbest\s)", re.IGNORECASE)
    neg_dict = {}
    pos_dict = {}

    for line in data:
        line = cleanse(line)
        line = remove_retweet_prefix(line)
        line = remove_at(line)
        line = remove_hashtag(line)
        line = remove_url(line)
        match = pattern.search(line)

        if match:
            p = get_polarity(line)
            doc = nlp(line)
            
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    name = ent.text.strip()
                    name = remove_apostrophe(name)
                    # not add to list if the entity is null or golden globe related
                    if p is None or name == '' or fuzz.ratio(name.lower(), 'golden globes') > 60:
                        continue
                    if p < 0:
                        if name in neg_dict:
                            neg_dict[name] += p
                        else:
                            neg_dict[name] = p
                    
                    if p > 0:
                        if name in pos_dict:
                            pos_dict[name] += p
                        else:
                            pos_dict[name] = p

#     pos = {key: value for key, value in persons_dic.items() if value > 10}
    # nominees = sorted(nominees.items(), key=lambda e: e[1], reverse=True)
    return pos_dict, neg_dict

pos_dict, neg_dict = find_names()

NameError: name 'remove_url' is not defined

In [79]:
# find the best dressed
print_top(pos_dict, 10)

# pprint.pprint(pos_dict)

[('Kate Hudson', 91.69873737373739),
 ('Lucy Liu', 43.47930839002267),
 ('Jennifer Lawrence', 39.818202260702265),
 ('Claire Danes', 39.555753968253974),
 ('Jessica Alba', 35.02333333333333),
 ('Anne Hathaway', 29.476388888888884),
 ('Jennifer Lopez', 24.532196969696955),
 ('Kerry Washington', 22.799531024531028),
 ('Kate', 21.302777777777777),
 ('Nicole Kidman', 19.533333333333335)]
list length: 578


In [82]:
# find the worst dressed
sorted_neg = sorted(neg_dict.items(), key=lambda e: e[1], reverse=False)
pprint.pprint(sorted_neg[0:10])
print(len(sorted_neg))
# pprint.pprint(neg_dict)

[('Sienna Miller', -14.383333333333333),
 ('Kate', -12.599999999999993),
 ('Kate Hudson', -11.9125),
 ('Lucy Liu', -11.85708874458874),
 ('Alexander McQueen', -11.555555555555548),
 ('Jennifer Lawrence', -9.566666666666666),
 ('Halle Berry', -7.711111111111111),
 ('NN45RgjF', -5.688888888888887),
 ('Nicole Kidman', -5.466666666666667),
 ('Anne Hathaway', -5.196153846153847)]
180


In [91]:
def get_name_to_reduce(nominees):
    names_clusters = []
    names = list(nominees.keys())

    for name in names:
        # each name starts as a cluster
        cluster = [name]
        names_to_reduce = names[:]
        names_to_reduce.remove(name)

        # one vs. all comparisons
        for i in names_to_reduce:
            ratio = fuzz.ratio(name.lower(), i.lower())
            # if similarity is larger than 75 or one name is contained in the other name
            if ratio > 75 or re.search(name, i, flags=re.IGNORECASE) or re.search(i, name, flags=re.IGNORECASE):
                cluster.append(i)

        # if multiple names are identified in one cluster
        if len(cluster) > 1:
            names_clusters.append(cluster)

    #     print(cluster)


    # sort clusters
    names_clusters.sort()
    # sort within each cluster
    names_clusters = ['|'.join(sorted(cluster)) for cluster in names_clusters]
    # remove overlaps
    names_clusters_reduced = [line.split('|') for line in list(set(names_clusters))]
    # sort by length from shortest to longest (merge from the shortest)
    names_clusters_reduced.sort(key=len)
#     print('\nnames clusters to merge:')
#     pprint.pprint(names_clusters_reduced)
#     print('\n')
    return names_clusters_reduced

In [102]:
# merge names using cluster
def reduce_names(nominees):
    reduced_nominees = nominees.copy()
    names_clusters_reduced = get_name_to_reduce(nominees)

    def weighted_freq(element):
        if element in reduced_nominees:
            return abs(reduced_nominees[element]) * len(element)
        else:
            return 0

    for cluster in names_clusters_reduced:
        # select the longest entity name
        selected_entity_name = max(cluster, key=weighted_freq)
        cluster.remove(selected_entity_name)
        # for names to be merged to the selected entity name
        for name in cluster:
            # if not deleted in previous cases, cumulate frequencies to the selected entity
            if name in reduced_nominees and selected_entity_name in reduced_nominees:
                reduced_nominees[selected_entity_name] += reduced_nominees[name]
                del reduced_nominees[name]
    return reduced_nominees

In [103]:
# merge similar names, find top 20
reduced_pos = reduce_names(pos_dict)
print_top(reduced_pos, 20)

[('Helena Bonham Carter Golden Globes Dress', 446.9144635126779),
 ('Jennifer Lawrence', 167.4609639434639),
 ('Anne Hathaway', 64.91242063492064),
 ('Lucy Liu', 62.54736394557823),
 ('Bradley Cooper', 38.42079725829725),
 ('Nicole Kidman', 33.43333333333334),
 ('Amy Poehler', 31.587240259740256),
 ('Halle Berry', 13.763029100529101),
 ('Lea Michele', 12.798125),
 ('Emily Blunt', 12.052380952380954),
 ('Michelle Dockery', 10.905555555555555),
 ('Tina Fey', 10.181565656565656),
 ('Miu Miu', 10.166666666666664),
 ('Tom Ford', 10.035714285714285),
 ('Katherine McPhee', 8.158333333333333),
 ('Kevin Dior', 7.5),
 ('Marion Cotillard', 6.494444444444444),
 ('Bill Clinton', 6.297619047619047),
 ('Megan Fox', 3.675),
 ('Helen Mirren', 3.3249999999999997)]
list length: 137


In [104]:
# merge similar names, find top 20
reduced_neg = reduce_names(neg_dict)
sorted_neg = sorted(reduced_neg.items(), key=lambda e: e[1], reverse=False)
pprint.pprint(sorted_neg[0:20])
print(len(sorted_neg))

[('Kate Hudson', -24.512499999999992),
 ('Lucy Liu', -17.05708874458874),
 ('Sienna Miller', -16.370833333333334),
 ('Alexander McQueen', -16.222222222222214),
 ('Jennifer Lawrence', -14.347619047619046),
 ('Halle Berry', -8.919444444444444),
 ('Anne Hathaway', -8.793881118881119),
 ('Eva Longoria', -8.641666666666666),
 ('NN45RgjF', -5.688888888888887),
 ('Nicole Kidman', -5.466666666666667),
 ('RT', -5.23670634920635),
 ('Lena Dunham', -4.641666666666667),
 ('Amy Poehler', -3.6802976190476184),
 ('PHOTOS', -3.1500000000000004),
 ("Daniel Day Lewis'", -2.9976190476190476),
 ('Jessica Chastain', -2.6025),
 ('Sorry', -2.4),
 ('Kristen Wiig', -2.386111111111111),
 ('Jodie Foster', -1.6714285714285713),
 ('Terrible Dress', -1.6666666666666667)]
109


In [109]:
# find the most controversial 
def get_sorted_names(nominees, f):
    sorted_nominees = sorted(nominees.items(), key=lambda e: e[1], reverse=f)
    names = [pair[0] for pair in sorted_nominees]
    return names

pos_names = get_sorted_names(reduced_pos, True)
neg_names = get_sorted_names(reduced_neg, False)

contro_dict = {}
for name in pos_names:
    if name in neg_names:
        contro_dict[name] = pos_names.index(name) + neg_names.index(name)

pprint.pprint(contro_dict)

{'Amy Poehler': 18,
 'Anne Hathaway': 8,
 'Don Cheadle': 92,
 'Halle Berry': 12,
 'Helen Hunt': 82,
 'Helen Mirren': 59,
 'Jennifer Lawrence': 5,
 'Lucy Liu': 4,
 'Megan Fox': 43,
 'Naeem Khan': 61,
 'Nicole Kidman': 14,
 'Quentin Tarantino': 116,
 'Tina Fey': 64}
